In [3]:
from google import genai
from dotenv import load_dotenv

load_dotenv()

client = genai.Client()
model_id = "gemini-2.0-flash"

#### select pdf files

In [8]:
file = "/Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/Nissan Motor Corporation-サステナビリティデータブック-2024.pdf"
filename = file.split(".")[-2].split("/")[-1]
pdf = client.files.upload(file=file, config={"display_name": filename})

In [9]:
file_size = client.models.count_tokens(model=model_id, contents=pdf)
print(f"File: {pdf.display_name} equalsto {file_size.total_tokens} tokens")

File: Nissan Motor Corporation-サステナビリティデータブック-2024 equalsto 197724 tokens


#### 4. コンペ用に動的にpydanticクラスを作成

In [100]:
from typing import Dict, Any, Optional
from pydantic import BaseModel, Field, create_model
import openai
import json

def create_pydantic_schema_prompt(questions: str) -> str:
    """"GPT-4に送信するプロンプトを生成する"""
    return f"""
以下の質問から、的確なポイントを押さえたPydanticのスキーマを生成してください。
必要に応じて、関連する追加フィールドも提案してください。
各フィールドには適切なデータ型とdescriptionをつけてください。

以下は出力の例です:

入力例:
日産自動車の2023年度の従業員の平均年収は約何万円でしょうか。
日産自動車のコーポレートパーパスは何ですか。
日産自動車の2023年度において、グローバル生産台数は何台か。

出力例：
{{
    "fields": [
        {{
            "name": "従業員平均年収",
            "type": "int",
            "description": "2023年度の日産自動車従業員の平均年収（単位：万円）",
        }},
        {{
            "name": "コーポレートパーパス",
            "type": "str",
            "description": "日産自動車の企業としての存在意義と目的を示す声明",
        }},
        {{
            "name": "グローバル生産台数",
            "type": "int",
            "description": "2023年度における世界全体での自動車生産台数",
        }},
    ]
}}
あなたの質問:
{questions}

出力形式:
{{
    "fields": [
        {{
            "name": "フィールド名",
            "type": "データ型",
            "description": "説明",
            "optional": true/false
        }},
        ...
    ]
}}

注意事項:
- 数値データは適切な型(int, float)を選択してください
- 必須/オプションを適切に設定してください
- descriptionは具体的で明確な説明を付けてください
- 関連する追加フィールドも積極的に提案してください
- 単位が必要な場合は、descriptionに明記してください
"""

def generate_pydantic_schema(questions: str) -> Dict[str, Any]:
    """GPT-4を使用してPydanticスキーマを生成する"""
    client = openai.OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "あなたはPydanticスキーマの専門家です。"},
            {"role": "user", "content": create_pydantic_schema_prompt(questions)},
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

def parse_schema_response(schema_str: str) -> Dict[str, Any]:
    """
    GPT-4からの応答をパースしてPythonの辞書に変換します。

    Args:
        schema_str (str): GPT-4から返された JSON 形式の文字列

    Returns:
        Dict[str, Any]: パースされたスキーマ辞書
    """
    try:
        # コードブロックのマーカー（```json や ```）を削除
        cleaned_str = schema_str.replace('```json', '').replace('```', '').strip()

        # JSON文字列をパース
        schema_dict = json.loads(cleaned_str)

        return schema_dict
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON format: {e}")
    except Exception as e:
        raise ValueError(f"Error parsing schema: {e}")

def create_dynamic_pydantic_class(schema_str: str, class_name: str = "Questionnaire") -> type[BaseModel]:
    schema_dict = parse_schema_response(schema_str)

    field_definitions = {}

    for field in schema_dict["fields"]:
        field_type = eval(field["type"])

        if field["optional"]:
            field_type = Optional[field_type]

        # OBJECT型でpropertiesが空の場合の対処
        if field["type"] == "Dict[str, int]" or field["type"] == "Dict[str, str]" or field["type"] == "Dict[str, Any]": #Dict型の場合
            if "properties" not in field or not field["properties"]:
                # propertiesが空の場合は、Any型を指定する
                field_type = Dict[str, Any] #変更
                print("field_typeを変更")

        field_definitions[field["name"]] = (
            field_type,
            Field(... if not field["optional"] else None,
                  description=field["description"]
                  )
            )
    return create_model(class_name, **field_definitions)

In [101]:
questions = "日産自動車の2023年度の従業員の平均年収は約何万円でしょうか。"

schema = generate_pydantic_schema(questions)

Questionnaire = create_dynamic_pydantic_class(schema)

In [105]:
Questionnaire.model_fields

{'従業員平均年収': FieldInfo(annotation=int, required=True, description='2023年度の日産自動車従業員の平均年収（単位：万円）'),
 'コーポレートパーパス': FieldInfo(annotation=str, required=True, description='日産自動車の企業としての存在意義と目的を示す声明'),
 'グローバル生産台数': FieldInfo(annotation=int, required=True, description='2023年度における世界全体での自動車生産台数'),
 '従業員数': FieldInfo(annotation=Union[int, NoneType], required=False, default=None, description='2023年度の日産自動車の全従業員数'),
 '売上高': FieldInfo(annotation=Union[float, NoneType], required=False, default=None, description='2023年度の日産自動車の総売上高（単位：億円）'),
 '営業利益': FieldInfo(annotation=Union[float, NoneType], required=False, default=None, description='2023年度の日産自動車の営業利益（単位：億円）')}

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field

class CompanyOverview(BaseModel):
    会社名: str = Field(..., description="会社名")
    本社所在地: str = Field(..., description="本社所在地")
    設立年月: str = Field(..., description="設立年月")
    代表者名: str = Field(..., description="代表者名")
    主要事業: str = Field(..., description="主要事業")
    上場市場: Optional[str] = Field(None, description="上場市場")
    証券コード: Optional[str] = Field(None, description="証券コード")
    URL: Optional[str] = Field(None, description="URL")

class FinancialHighlights(BaseModel):
    売上高: int = Field(..., description="売上高")
    営業利益: int = Field(..., description="営業利益")
    経常利益: int = Field(..., description="経常利益")
    当期純利益: int = Field(..., description="当期純利益")
    総資産: int = Field(..., description="総資産")
    自己資本: int = Field(..., description="自己資本")
    ROE: float = Field(..., description="ROE")
    EPS: float = Field(..., description="EPS")

class ESG(BaseModel):
    環境: Optional[str] = Field(None, description="環境に関する取り組み")
    社会: Optional[str] = Field(None, description="社会に関する取り組み")
    ガバナンス: Optional[str] = Field(None, description="ガバナンスに関する取り組み")

class ShareholderInformation(BaseModel):
    株主数: int = Field(..., description="株主数")
    発行済株式数: int = Field(..., description="発行済株式数")
    配当金: float = Field(..., description="配当金")
    株価情報: Optional[str] = Field(None, description="株価情報")

class ManagementStrategy(BaseModel):
    ビジョン: Optional[str] = Field(None, description="ビジョン")
    中期経営計画: Optional[str] = Field(None, description="中期経営計画")
    事業戦略: Optional[str] = Field(None, description="事業戦略")

class OrganizationInformation(BaseModel):
    組織図: Optional[str] = Field(None, description="組織図")
    従業員数: int = Field(..., description="従業員数")


class IntegratedReport(BaseModel):
    company_overview: CompanyOverview
    financial_highlights: FinancialHighlights
    esg: ESG
    shareholder_information: ShareholderInformation
    management_strategy: ManagementStrategy
    organization_information: OrganizationInformation
    other_information: Optional[str] = Field(None, description="その他の情報")
    questionnaire: Questionnaire

In [103]:
prompt = "以下のPDFファイルをGemini Flash APIで読み取り、Pydanticモデルに基づいて構造化されたJSON形式で出力してください。"
response = client.models.generate_content(model=model_id, contents=prompt, config={"response_mime_type": "application/json", "response_schema": IntegratedReport})
print(response.text)

{
  "company_overview": {
    "主要事業": "自動車製造",
    "代表者名": "内田 誠",
    "会社名": "日産自動車株式会社",
    "本社所在地": "神奈川県横浜市",
    "設立年月": "1933年12月",
    "URL": "https://www.nissan-global.com/JP/",
    "上場市場": "東京証券取引所",
    "証券コード": "7201"
  },
  "esg": {
    "ガバナンス": "企業統治体制の強化、コンプライアンスの徹底",
    "環境": "電動化技術の開発、CO2排出量削減",
    "社会": "多様な人材の活躍支援、地域社会への貢献"
  },
  "financial_highlights": {
    "EPS": 45.67,
    "ROE": 8.2,
    "営業利益": 5000,
    "売上高": 100000,
    "当期純利益": 3000,
    "経常利益": 5500,
    "総資産": 150000,
    "自己資本": 60000
  },
  "management_strategy": {
    "ビジョン": "革新的なモビリティで、持続可能な社会に貢献する",
    "中期経営計画": "Nissan Ambition 2030",
    "事業戦略": "電動化、知能化、コネクテッド化を推進"
  },
  "organization_information": {
    "従業員数": 130000,
    "組織図": null
  },
  "questionnaire": {
    "グローバル生産台数": 3400000,
    "コーポレートパーパス": "人々の生活を豊かにする",
    "従業員平均年収": 850,
    "営業利益": 5000.00,
    "売上高": 100000.00,
    "従業員数": 130000
  },
  "shareholder_information": {
    "株主数": 350000,
    "発行済株式数": 4000000000,
    "配当金": 2

In [110]:
import json

def create_context(model_fields, data_str):
    """
    Pydanticモデルのフィールド情報とデータを組み合わせてコンテキストを生成する

    Args:
        model_fields (dict): Pydanticモデルのフィールド情報
        data_str (str): JSON形式の文字列データ

    Returns:
        str: 生成されたコンテキスト
    """
    # JSON文字列をパース
    try:
        data = json.loads(data_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON format: {e}")

    # クエスチョネアデータの抽出
    questionnaire_data = data.get('questionnaire', {})

    # コンテキストの構築
    context_parts = []

    # 各フィールドについて処理
    for field_name, field_info in model_fields.items():
        if field_name in questionnaire_data:
            value = questionnaire_data[field_name]
            description = field_info.description
            context_parts.append(f"{description}: {value}")

    # コンテキストの結合
    context = "コンテキスト：\n" + "\n".join(context_parts)

    return context

In [111]:
context = create_context(Questionnaire.model_fields, response.text)

In [112]:
context

'コンテキスト：\n2023年度の日産自動車従業員の平均年収（単位：万円）: 850\n日産自動車の企業としての存在意義と目的を示す声明: 人々の生活を豊かにする\n2023年度における世界全体での自動車生産台数: 3400000\n2023年度の日産自動車の全従業員数: 130000\n2023年度の日産自動車の総売上高（単位：億円）: 100000.0\n2023年度の日産自動車の営業利益（単位：億円）: 5000.0'

### コードのテスト

In [113]:
from rag_code.create_pydantic_schema import CreatePydanticSchema
from rag_code.create_integrated_pydantic_model import create_integrated_report_model
from rag_code.gemini_flash_pdf_output import response_gemini
from rag_code.gemini_flash_pdf_output import create_context


%load_ext autoreload
%autoreload 2

In [114]:
filepath = "/Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/Nissan Motor Corporation-サステナビリティデータブック-2024.pdf"
search = CreatePydanticSchema(filepath)
question = "日産自動車の2023年度の従業員の平均年収は約何万円でしょうか。"
schema_str = search.generate_pydantic_schema(question)
print(schema_str)
Questionnair = search.create_dynamic_pydantic_class(schema_str)
IntegratedReport = create_integrated_report_model(Questionnair)
response = response_gemini(question, IntegratedReport)
context = create_context(Questionnair.model_fields, response)

{
    "fields": [
        {
            "name": "従業員平均年収",
            "type": "int",
            "description": "2023年度の日産自動車従業員の平均年収（単位：万円）",
            "optional": false
        },
        {
            "name": "コーポレートパーパス",
            "type": "str",
            "description": "日産自動車の企業としての存在意義と目的を示す声明",
            "optional": false
        },
        {
            "name": "グローバル生産台数",
            "type": "int",
            "description": "2023年度における世界全体での自動車生産台数",
            "optional": false
        },
        {
            "name": "設立年",
            "type": "int",
            "description": "日産自動車が設立された年",
            "optional": true
        },
        {
            "name": "本社所在地",
            "type": "str",
            "description": "日産自動車の本社が所在する場所",
            "optional": true
        }
    ]
}


In [115]:
context

'コンテキスト：\n2023年度の日産自動車従業員の平均年収（単位：万円）: 800\n日産自動車の企業としての存在意義と目的を示す声明: 人々の生活を豊かにする\n2023年度における世界全体での自動車生産台数: 3500000\n日産自動車が設立された年: 1933\n日産自動車の本社が所在する場所: 神奈川県横浜市'

In [1]:
contex = 'コンテキスト：\n2023年度の日産自動車従業員の平均年収（単位：万円）: 800\n日産自動車の企業としての存在意義と目的を示す声明: 人々の生活を豊かにする\n2023年度における世界全体での自動車生産台数: 3500000\n日産自動車が設立された年: 1933\n日産自動車の本社が所在する場所: 神奈川県横浜市'